# Azure ML SDK v2 (v1.15)-  Notebook to login in different ways
- If you have installed: "azure-enterprise-scale-ml\environment_setup\user_dev_env_install\AzureML_v1_55_and_v2_1_15"
    Then you dont need to run below PIP
- If you have installed: "azure-enterprise-scale-ml\environment_setup\user_dev_env_install\AzureML_v1_55"
    - Then you need to run below PIP (3 cells)
        - azure-ai-ml, azure-identity, python-dotenv


In [ ]:
%pip install azure-ai-ml

In [ ]:
%pip install azure-identity

In [ ]:
%pip install -U python-dotenv

In [ ]:
subscription_id="guid"
resource_group_name="name"
workspace_name="name"

project_numnber="1"
compute_instance_sku = "Standard_DS11_v2" # "STANDARD_NC24RS_V3", "Standard_D13_v2"
compute_instance_suffix = "01"

# Option A) Default credential

In [ ]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group_name, workspace_name
)

# Option B) Interactive login (forced)

In [ ]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

credential = InteractiveBrowserCredential()

# Option C) Service Principal login - with client secret in .env file
- Create an .env file with the below properties: 

AZURE_CLIENT_ID=aGuid

AZURE_TENANT_ID=aGuid

AZURE_CLIENT_SECRET=aSecret



In [ ]:
import os
from dotenv import load_dotenv

print("Loading environment variables from .env file")
load_dotenv(".env")

from azure.identity import DefaultAzureCredential

credential = DefaultAzureCredential()
# Check if given credential can get token successfully.
credential.get_token("https://management.azure.com/.default")

# After option a,b or c the below LOGIN code can be used

In [ ]:
from azure.ai.ml import MLClient
try:
    ml_client = MLClient.from_config(credential=credential)
except Exception as ex:
    # NOTE: Update following workspace information to contain
    #       your subscription ID, resource group name, and workspace name
    client_config = {
        "subscription_id": subscription_id,
        "resource_group": resource_group_name,
        "workspace_name": workspace_name,
    }

    # write and reload from config file
    import json, os

    config_path = "../.azureml/config.json"
    os.makedirs(os.path.dirname(config_path), exist_ok=True)
    with open(config_path, "w") as fo:
        fo.write(json.dumps(client_config))
    ml_client = MLClient.from_config(credential=credential, path=config_path)

print(ml_client)

In [ ]:
# Compute Instances need to have a unique name across the region.
# Here we create a unique name with current datetime
from azure.ai.ml.entities import ComputeInstance, AmlCompute
import datetime

ci_basic_name = "p00"+project_numnber+"-m01-xyz-dev-ci"+compute_instance_suffix # + datetime.datetime.now().strftime("%Y%m%d%H%M")

ci_basic = ComputeInstance(name=ci_basic_name, size=compute_instance_sku)
ml_client.begin_create_or_update(ci_basic).result()

# Note that idle_time_before_shutdown has been deprecated.
ComputeInstance(name=ci_basic_name, size=compute_instance_sku, idle_time_before_shutdown_minutes="30")